In [38]:
import pandas as pd
import requests
from config import api_key

url="http://api.openweathermap.org/data/2.5/weather"



In [23]:
r=requests.get(url=url, params={
    "q": "london",
    "appid":api_key
    })

In [24]:
r.json()

{'coord': {'lon': -0.13, 'lat': 51.51},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 292.87,
  'feels_like': 289.72,
  'temp_min': 292.04,
  'temp_max': 294.26,
  'pressure': 1006,
  'humidity': 64},
 'visibility': 10000,
 'wind': {'speed': 5.7, 'deg': 240},
 'clouds': {'all': 75},
 'dt': 1600796961,
 'sys': {'type': 1,
  'id': 1414,
  'country': 'GB',
  'sunrise': 1600753627,
  'sunset': 1600797551},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

In [39]:
cities=pd.read_csv("worldcities.csv")

name=cities["City"]

name=name[::30]

city_list=name.head(500).reset_index(drop=True).to_frame()


city_list.head()



,City
0,andorra la vella
1,gazni
2,tagab
3,fierze
4,patos


In [40]:
def extract(data):
    return {
        "city": data["name"],
        "lat": data["coord"]["lat"],
        "lon": data["coord"]["lon"],
        "temp_k": data["main"]["temp"],
        "humidity": data["main"]["humidity"],
        "cloudiness": data["clouds"],
        "wind_speed_mph": data["wind"]["speed"]
    }

In [41]:
city_results=[]

In [43]:
for city in city_list:
    data=requests.get(url=url, params={
    "q": city,
    "appid":api_key
    }).json()
    results=extract(data)
    city_results.append(results)

In [44]:
df=pd.DataFrame(city_results)
df

,city,lat,lon,temp_k,humidity,cloudiness,wind_speed_mph
0,City,-35.28,149.13,280.05,75,{'all': 5},4.1
1,City,-35.28,149.13,279.91,75,{'all': 5},4.1
